# Data Cleaning

Even though we now have all our extracted data, there are still null values and string and object format columns. Because of that, we had to clean our data.

We start by importing relevant libraries and the dataframes we obtained in the feature extraction step.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_pyradiomics = pd.read_csv('pyradiomics_features_fixed.csv', index_col=False)
df_pylidc = pd.read_csv('pylidc_features_fixed.csv', index_col=False)

### Binary Classification

Before merging both dataframes, we decided to tackle this problem using a binary target, malign and benign.

#### Default distribution

We start by looking at how our target's distribution by the different labels.

In [ ]:
unique_labels, counts = np.unique(df_pylidc.malignancy, return_counts=True)
plt.bar(unique_labels, counts)
plt.xlabel('Class label')
plt.ylabel('Number of samples')
plt.title('Class distribution in dataset')
plt.show()

print(unique_labels, counts)

The first step for turning our target binary is merging similar classes:

- -1  -> 'Highly Unlikely'(1) and 'Moderately Unlikely'(2)
- 0  -> 'Indeterminate'(3)
- 1  -> 'Moderately Suspicious'(4) and 'Highly Suspicious'(5)

In [ ]:
for i in range(len(df_pylidc)):
    if df_pylidc.at[i,'malignancy'] == 1 or df_pylidc.at[i,'malignancy'] == 2:
        df_pylidc.at[i,'malignancy'] = -1
    elif df_pylidc.at[i,'malignancy'] == 4 or df_pylidc.at[i,'malignancy'] == 5:
        df_pylidc.at[i,'malignancy'] = 1
    else:
        df_pylidc.at[i,'malignancy'] = 0

In [ ]:
unique_labels, counts = np.unique(df_pylidc.malignancy, return_counts=True)
plt.bar(unique_labels, counts)
plt.xlabel('Class label')
plt.ylabel('Number of samples')
plt.title('Class distribution in dataset')
plt.show()

print(unique_labels, counts)

Next, we transform '0' labeled examples into the most common class within each patient. For that we start by defining a function that returns the most common class.

In [ ]:
def most_common_class(group):
    mode_class = group['malignancy'].mode()
    if not mode_class.empty:
        return mode_class.iloc[0]
    else:
        return 0

Then, we create a dictionary to store the most common class for each patient and change our dataframe.

In [ ]:
most_common_per_patient = df_pylidc.groupby('patient_id').apply(most_common_class).to_dict()

df_pylidc['malignancy'] = df_pylidc.apply(lambda row: most_common_per_patient.get(row['patient_id'], row['malignancy']) if row['malignancy'] == 0 else row['malignancy'], axis=1)

In [ ]:
unique_labels, counts = np.unique(df_pylidc.malignancy, return_counts=True)
plt.bar(unique_labels, counts)
plt.xlabel('Class label')
plt.ylabel('Number of samples')
plt.title('Class distribution in dataset')
plt.show()

print(unique_labels, counts)

Finally, we remove the remaining patients with target '0' as majority. For that, we group the dataframe by 'patient_id', count the ocurrences of each diagnosis and identify the ones we want to remove.

In [ ]:
diagnosis_counts = df_pylidc.groupby('patient_id')['malignancy'].value_counts().unstack(fill_value=0)

majority_0_patients = diagnosis_counts.idxmax(axis=1) == 0

df_pylidc = df_pylidc[~df_pylidc['patient_id'].isin(majority_0_patients[majority_0_patients].index)]

In [ ]:
unique_labels, counts = np.unique(df_pylidc.malignancy, return_counts=True)
plt.bar(unique_labels, counts)
plt.xlabel('Class label')
plt.ylabel('Number of samples')
plt.title('Class distribution in dataset')
plt.show()

print(unique_labels, counts)

In [ ]:
df_pylidc.to_csv('final_pylidc_extraction.csv', sep=',', index=False)

### Features Merge

The next step, after having the target in the format we wanted, is to merge both datasets.

In [ ]:
print(df_pylidc.shape)
print(df_pyradiomics.shape)

In [ ]:
df = df_pyradiomics.merge(df_pylidc, left_on='id', right_on='Id')
df = df.drop(columns=['Id'])
df.shape

### Columns removal

#### Null Values and Constants

We removed all the columns that are constants in the dataframe or that contained null values.

In [ ]:
removed = 0
for col in df.columns:
    if len(df[col].unique()) == 1:
        df = df.drop(columns=col)
        removed += 1

for col in df.columns:
    null_count = df[col].isnull().sum()
    if null_count > 0:
        df = df.drop(columns=col)
        removed += 1

We were left with the following columns.

In [ ]:
print(df.shape)
print(f"Total number of columns deleted: {removed}")

We took then a look at the number of individual patients in the training set and the resulting distribution of the target class.

In [ ]:
len(df['patient_id'].unique())

In [ ]:
unique_labels, counts = np.unique(df.malignancy, return_counts=True)
plt.bar(unique_labels, counts)
plt.xlabel('Class label')
plt.ylabel('Number of samples')
plt.title('Class distribution in dataset')
plt.show()

print(unique_labels, counts)

#### String and Object Types

We need to be careful when working with string and object types, so we start by finding out how many we had, and which they were.

In [ ]:
df.dtypes.value_counts()

In [ ]:
df_groupby_dtypes = df.columns.to_series().groupby(df.dtypes).groups

for key, value in df_groupby_dtypes.items():
    if key == 'object':
        print(value)

We then gathered the object columns (except for ids) and removed them.

Some contained coordinate tuples, which we below converted into new columns.

In [ ]:
object_columns = [
    'diagnostics_Mask-original_Spacing',
    'diagnostics_Mask-original_Size',
    'diagnostics_Mask-original_BoundingBox',
    'diagnostics_Mask-original_CenterOfMassIndex',
    'diagnostics_Mask-original_CenterOfMass',
    'diagnostics_Image-interpolated_Spacing',
    'diagnostics_Image-interpolated_Size',
    'diagnostics_Mask-interpolated_Spacing',
    'diagnostics_Mask-interpolated_Size',
    'diagnostics_Mask-interpolated_BoundingBox',
    'diagnostics_Mask-interpolated_CenterOfMassIndex',
    'diagnostics_Mask-interpolated_CenterOfMass',
    'diagnostics_Mask-original_Hash',
    'original_shape_Elongation', 
    'original_shape_Flatness',
    'original_shape_LeastAxisLength', 
    'original_shape_MajorAxisLength',
    'original_shape_MinorAxisLength',
    ]

object = df[object_columns]
df = df.drop(columns = object_columns)

To treat string type columns '(x, y, z)' we iterated and separated them into separate columns x, y and z.

In [ ]:
for j in range(0, 12):
    col = object[object_columns[j]]
    comma_count = col.iloc[0].count(',')

    # define new column names 
    list = []
    for i in range(comma_count + 1):
        name = object_columns[j] + "_" + str(i)
        list.append(name)

    for k in range(len(object)):
        s = col.iloc[k]

        # split the column into separate columns
        s = s.strip('()')
        values = [float(idx) for idx in s.split(', ')]
        
        for k1 in range(len(list)):
            object[list[k1]] = values[k1]

    object = object.drop(columns= object_columns[j])

Then we dropped the remaining columns with questionable data, such as hash values or imaginary values, for example.

In [ ]:
object = object.drop(columns= object.columns[0:6])
object.head()

When merging the new columns with the dataframe, we can see we no longer have object types asides from 'id' and 'patient_id'.

In [ ]:
df = pd.concat([df, object], axis=1)
print(df.dtypes.value_counts())
print(df.shape)

#### Low Correlation

We checked the correlation of our features with the target and decided to removed the ones with correlation lower than 0.05.

In [ ]:
index = df[['id', 'patient_id']]
features = df.drop(columns=['id', 'patient_id'])

In [ ]:
correlations_with_target = features.corr()['malignancy']
features_to_remove = correlations_with_target[abs(correlations_with_target) < 0.05].index
df_filtered = features.drop(columns=features_to_remove)

df_filtered.shape

In [ ]:
df = pd.concat([index, df_filtered], axis=1) 
df.shape

We saved our final CSV file, which will be used for model training and testing.

In [ ]:
df.to_csv('data.csv', sep=',', index=False)